In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
dataset=pd.read_csv("Churn_Modelling.csv")

In [ ]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
X=dataset.iloc[:,3:13]#independent features
Y=dataset.iloc[:,13]#dependent features

In [ ]:
#Create dummpies varibale
geography=pd.get_dummies(X['Geography'],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [ ]:
#Concat the dataframe

X=pd.concat([X,geography,gender],axis=1)

In [ ]:
#Drop uncessary column
X=X.drop(['Geography','Gender'],axis=1)

In [ ]:
#Splitting the datsets into training and test data sets
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [ ]:
#import keras libaries and package
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout


In [ ]:
#Modifying the code(Should apply this section of code always)
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model(layers,activation):
  model=Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  

  model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
model=KerasClassifier(build_fn=create_model,verbose=0)

In [ ]:
#Hyperparameter Tunning using GridSearchCV
layers=[[20],[40,20],[45,30,15]]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)


In [ ]:
grid_result=grid.fit(X_train,Y_train)

In [ ]:
print(grid_result.best_score_,grid_result.best_params_)

0.8547500014305115 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [ ]:
#Initializing the ANN
classifier=Sequential()

In [ ]:
#Adding the input Layer and first hidden layer
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=11))
classifier.add(Dropout(0.3))

In [ ]:
#Adding the second hidden Layer
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dropout(0.4))

In [ ]:
#Adding the output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [ ]:
classifier.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Compiling the ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#Fitting the ANN to thr training set
model_history=classifier.fit(X_train,Y_train,validation_split=0.33,batch_size=10,epochs= 100)

Epoch 1/100
536/536 [==============================] - 3s 2ms/step - loss: 0.8891 - accuracy: 0.5450 - val_loss: 0.5379 - val_accuracy: 0.7955
Epoch 2/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5641 - accuracy: 0.7753 - val_loss: 0.4984 - val_accuracy: 0.7955
Epoch 3/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5037 - accuracy: 0.7991 - val_loss: 0.4805 - val_accuracy: 0.7955
Epoch 4/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5051 - accuracy: 0.7826 - val_loss: 0.4673 - val_accuracy: 0.7955
Epoch 5/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5035 - accuracy: 0.7912 - val_loss: 0.4618 - val_accuracy: 0.7955
Epoch 6/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4853 - accuracy: 0.7909 - val_loss: 0.4557 - val_accuracy: 0.7955
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4730 - accuracy: 0.7971 - val_loss: 0.4432 - val_accuracy: 0.7955

In [ ]:
#predicting the test result sets
y_pred=classifier.predict(X_test)
y_pred=(y_pred>0.5)

In [ ]:
#Making the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)

In [ ]:
#Calculate the accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(Y_test,y_pred)